In [47]:
(720000-144000)/420

1371.4285714285713

In [48]:
(720000-144000)*(1.0829**(1/12)-1)+(720000-144000)/420


5206.98870658821

In [49]:
(1.0872)**(1/12)-1

0.006991458915456494

In [50]:
import pandas as pd

# Imputs

In [51]:
valor_imovel=720000
entrada_pp=0.2
n_parcelas=420                      # 35 anos
juros_nominal=0.0799                # a.a

# Custos Iniciais

In [52]:
custo_itbi=0.03*valor_imovel
custo_escritura=3000
custo_registro=2000
custo_avaliacao=4000


In [53]:
juros_efetivo=((1+juros_nominal/12)**(12)-1)            # Efeito anual dos juros sobre juros mensais.  

valor_entrada=valor_imovel*entrada_pp
valor_finaciado=valor_imovel-valor_entrada

Encargos_finaciamento=130

In [54]:
amortizacao=valor_finaciado/n_parcelas
saldo_devedor=valor_finaciado

# Cálculo do juros em função do mês
def parcela(n):
    saldo_devedor=valor_finaciado-n*amortizacao
    juros=saldo_devedor*juros_nominal/12
    return amortizacao+juros+Encargos_finaciamento

df=pd.DataFrame(columns=['parcela'],index=range(0,n_parcelas))

for i in range(0,n_parcelas):
    df.iloc[i,0]=parcela(i)
df

,parcela
0,5336.628571
1,5327.497143
2,5318.365714
3,5309.234286
4,5300.102857
...,...
415,1547.085714
416,1537.954286
417,1528.822857
418,1519.691429


# Custos fixos

In [56]:
condominio=704
iptu=1530/12
luz=120

# Custos com reforma

In [57]:
obra=95000
prazo_obra=4                # Prazo da obra em mesês